In [1]:
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook

In [2]:
import numpy as np

In [3]:
%%time

book = load_workbook("Find Journal Lines - NMSS 2020-08-11 13_55 PDT.xlsx")

C:\Users\jeremy.wendt\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Wall time: 2min 28s


In [4]:
current_month = 10

In [5]:
ws = book["Sheet1"]

In [6]:
num_rows = ws.max_row

curr_row = 37

print(num_rows)

254212


In [7]:
titles = [cell.value for cell in ws[37]]

In [8]:
print(titles)

['Journal', 'Journal Number', 'Journal Sequence Number', 'Journal Memo', 'Operational Transaction', 'Originated by', 'Approved by', 'Company', 'Intercompany Initiating Company', 'Status', 'Accounting Date', 'Posted Date', 'Source', 'Ledger', 'Currency', 'Ledger Account', 'Ledger Debit Amount', 'Ledger Credit Amount', 'Line Memo', 'External Reference ID', 'Spend Category', 'Revenue Category', 'Cost Center', 'Fund', 'Gift', 'Program', 'Project', 'Region', 'Location', 'Fundraising Type', 'InKind', 'State', 'Bank Account', 'Intercompany Affiliate', 'Supplier', 'Customer', 'Employee', 'Match ID']


In [9]:
%%time
data = []

while curr_row < num_rows:
    curr_row += 1
    row = {}
    for i in range(len(titles)):
                row[titles[i]] = ws.cell(row=curr_row, column=i+1).value
    data.append(row)
        

Wall time: 23.5 s


In [10]:
# Need to add column for In-Kind
for row in data:
    row['Debit-Credit'] = row['Ledger Debit Amount'] - row['Ledger Credit Amount']

In [11]:
for row in data:
    number = row['Ledger Account'][:1]
    if number == str(5):
        row['Rev/Exp/DBD'] = "Expense"
    elif number == str(4):
        row['Rev/Exp/DBD'] = "Revenue"
    elif number == str(6):
        row['Rev/Exp/DBD'] = "DBD"
    else:
        row['Rev/Exp/DBD'] = "Other"    

In [12]:
import datetime

for row in data:
    row['Month'] = row['Accounting Date'].month
    
print(data[600]['Month'])

10


In [13]:
book2 = load_workbook("Ledger_Account_Table.xlsx")

ws2 = book2["Sheet1"]

In [14]:
curr_row = 0


data2 = []
titles2 = []


while curr_row < num_rows:
        curr_row += 1
        if not titles2:
            titles2 = [cell.value for cell in ws2[curr_row]]
            continue
        row = {}
        for i in range(len(titles2)):
            row[titles2[i]] = ws2.cell(row=curr_row, column=i+1)
        data2.append(row)
        
print(data2[2]['Ledger Account'].value)

4002:Event Sponsorships


In [15]:
data3 = {}

for row in data2:
    data3[row['Ledger Account'].value]=row['In/Out'].value



In [16]:
for row in data:
    row['In/Out'] = data3.get(row['Ledger Account'], "")

In [17]:
Spend_Cats = []

for i in range(len(data)):
    if data[i]['In/Out']=='In':
        Spend_Cats.append(data[i]['Spend Category']) 
    
spendcats = list(set(Spend_Cats))
print(spendcats)
    

['Print Media', 'Contributed Services Expense', 'Audit Fees', 'Email Communications', 'Postage', 'Depreciation Expense', 'Meetings, Conferences, Conventions', 'Office Equipment Expense', 'DBD Event Supplies', 'Supplies General Office', 'DBD Volunteer Incentives', 'DBD Entertainment', 'DFA Critical ST Needs', 'DBD Other Expense', 'Other Professional Fees', 'DFA Health and Wellness', 'Lodging', 'Research Grant Expense', 'Joint Cost Postage', 'Miscellaneous Expense', 'Computer Equipment Expense', 'DFA Mental Health Support', 'DBD Promotional Items', 'Telephone Expense', 'Miscellaneous Interest Expense', 'Joint Cost Professional Fees', 'Bad Debt Expense', 'DBD Sanitation', 'Traditional Advertising & Media', 'Express Mail Delivery', 'Airfare', 'Meeting Entertainment', 'Photographs', 'Business Insurance & Casualty Expense', 'DBD Travel Expense', 'DBD Participant Incentives', 'Website-Web Presence', 'Audio & Visual Content Creation', 'DBD Prizes', 'Bank Fees', 'Repairs & Maintenance', 'DBD Te

In [18]:
FYmonths = [10,11,12,1,2,3,4,5,6,7,8,9]

print(FYmonths)

[10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [19]:
Cost_centers = []

for i in range(len(data)):
    Cost_centers.append(data[i]['Cost Center']) 
    
myset = list(set(Cost_centers))
print(myset)

['CC019 Corporate Relations', 'CC033 Community Leadership', 'CC008 Legal and Risk', 'CC020 Do It Yourself (DIY)', 'CC005 Facilities', 'CC018 Direct Marketing\xa0', 'CC003 ELT', 'CC029 Professional Education', 'CC025 Other Special Events', 'CC032 Services Resources', 'CC010 Digital Marketing', 'CC021 Challenge Walk', 'CC015 Development', 'CC011 Brand Management', 'CC028 MS Navigator', 'CC034 Community Engagement', 'CC006 Human Resources', 'CC024 Walk MS', None, 'CC007 Volunteer Engagement', 'CC022 Leadership Events', 'CC012 Technology', 'CC013 Strategic Finance', 'CC001 Research', 'CC017 Tools and Resources', 'CC014 Finance Operations', 'CC035 Event Production', 'CC023 Bike MS', 'CC002 Advocacy', 'CC009 Marketing', 'CC036 Fundraising Support', 'CC026 MuckFest MS', 'CC016 Individual Giving', 'CC004 Operations', 'CC027 Programs & Services']


In [20]:
def create_department(Cost_Center):
    listname = []
    for key, row in enumerate(data):
        if row['Cost Center'] == Cost_Center and row['In/Out']=="In":
            listname.append(row)    
    return listname

In [21]:
Volunteer_Engagement = create_department('CC007 Volunteer Engagement')
HR = create_department('CC006 Human Resources')
Technology = create_department('CC012 Technology')
FinanceOps = create_department('CC014 Finance Operations')
Facilities = create_department('CC005 Facilities')
Operations = create_department('CC004 Operations')
Legal = create_department('CC008 Legal and Risk')
    

In [22]:
def filter_write(SC, dept, month):
    total = 0
    for key,row in enumerate(dept):
        if row['Spend Category'] == SC and row['Month'] == month:
            total += row['Debit-Credit']
    return total




In [23]:
def filter_Rev(GL, dept, month):
    total = 0
    for key,row in enumerate(dept):
        if row['Ledger Account'] == GL and row['Month'] == month:
            total += row['Debit-Credit']
    return total

In [29]:
file = 'Dept_Template.xlsx'

wb = openpyxl.load_workbook(filename=file)
ws3 = wb['Sheet1']    


In [30]:
def copy_revenue(Department):
    for i in range(len(FYmonths)):   
        for j in range(ws3.max_row):
            if ws3.cell(row = j+1, column = 1).value == 1 or ws3.cell(row = j+1, column = 2).value == None:
                continue      
            elif ws3.cell(row = j+1, column = 2).value == 'Total Expense':
                break 
            else:
                ws3.cell(row = j+1, column = i+3).value = filter_Rev(ws3.cell(row=j+1, column = 2).value, Department, FYmonths[i]) 


In [31]:
def copy_expenses(Department):
    for i in range(len(FYmonths)):   
        for j in range(18, ws3.max_row):
            if ws3.cell(row = j+1, column = 1).value == 1 or ws3.cell(row = j+1, column = 2).value == None:
                continue      
            elif ws3.cell(row = j+1, column = 2).value == 'Total Expense':
                break 
            else:
                ws3.cell(row = j+1, column = i+3).value = filter_write(ws3.cell(row=j+1, column = 2).value, Department, FYmonths[i]) 


In [32]:
def copy_data_save(Department, name, month):
    copy_revenue(Department)
    copy_expenses(Department)
    ws3.cell(row = 3, column = 3).value = month
    ws3.cell(row = 4, column = 3).value = name
    wb.save(name+".xlsx")
    

In [33]:
copy_data_save(Facilities, "Facilities", current_month)
copy_data_save(HR, "Human_Resources", current_month)
copy_data_save(FinanceOps, "Finance", current_month)
copy_data_save(Operations, "Operations", current_month)
copy_data_save(Legal, "Legal", current_month)